In [4]:
import sys
from pathlib import Path
sys.path.append(str(Path.cwd() / "notebooks"))  # so we can import _utils from notebooks/

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from _utils import get_data_dir

DATA_DIR = get_data_dir()
DATA_DIR

WindowsPath('C:/Users/AdamR/OneDrive/UCSB/VIU/HonorsThesis/data')

In [13]:


# ---------------------------------------------------------------------
# 0. Setup
# ---------------------------------------------------------------------
import dis


DATA_DIR = get_data_dir()
ROOT = DATA_DIR          # matches your humans.py
conds = ["50_50", "80_20", "100_0"]

# ---------------------------------------------------------------------
# 1. Build human_master exactly like humans.py
# ---------------------------------------------------------------------
human_master = pd.concat(
    [pd.read_csv(ROOT / cond / "human_data.csv") for cond in conds],
    ignore_index=True
)

# Map condition codes to your nicer names
human_master["condition"] = human_master["condition"].map({
    "condition_1": "80_20",
    "condition_2": "50_50",
    "condition_3": "100_0"
})

# Keep / order desired columns (from humans.py)
cols = [
    "stimID", "condition", "response", "side_selected", "cue_points",
    "line1_angle", "line2_angle", "valid_cue", "TP", "participantID"
]
human_master = human_master[cols]

# Binary decision from confidence rating
human_master["decision"] = (human_master["response"] >= 4).astype(int)

# New columns for the master file
human_master["type"] = "human"
# Treat the 1–6 response as human confidence
human_master["confidence"] = human_master["response"]

# ---------------------------------------------------------------------
# 2. Build Gemini 2.5 Pro rows like in models.py
#    (only the angle-estimation model)
# ---------------------------------------------------------------------
def parse_gemini(resp):
    """
    Parse 'a1, a2, present/absent' from GPT_response, like in models.py.
    """
    a1, a2, dec = [p.strip() for p in str(resp).split(",")]
    return float(a1), float(a2), dec.lower()

gemini_frames = []

for cond in conds:
    base = ROOT / cond

    # angle-estimation Gemini CSV
    g = pd.read_csv(base / "angle_estimations" / "gemini-2.5-pro.csv")

    # Split GPT_response into est1, est2, and response (present/absent)
    g[["est1", "est2", "response"]] = pd.DataFrame(
        g["GPT_response"].apply(parse_gemini).tolist(),
        index=g.index
    )

    g["stimID"] = g["image_id"]
    g["condition"] = cond
    g["participantID"] = "gemini-2.5-pro"
    g["decision"] = g["response"].map({"present": 1, "absent": 0})

    # Match human columns + keep est1/est2
    g = g[[
        "stimID", "condition", "response", "side_selected", "cue_points",
        "line1_angle", "line2_angle", "valid_cue", "TP",
        "participantID", "decision", "est1", "est2"
    ]]

    gemini_frames.append(g)

gemini = pd.concat(gemini_frames, ignore_index=True)

# Add 'type' and 'confidence' as requested
gemini["type"] = "model"
gemini["confidence"] = np.nan   # no confidence for Gemini

# ---------------------------------------------------------------------
# 3. Combine into a single master CSV
# ---------------------------------------------------------------------
project_master = pd.concat(
    [human_master, gemini],
    ignore_index=True,
    sort=False
)

out_path = ROOT / "231_project_master.csv"
project_master.to_csv(out_path, index=False)

display(project_master)


,stimID,condition,response,side_selected,cue_points,line1_angle,line2_angle,valid_cue,TP,participantID,decision,type,confidence,est1,est2
0,100,50_50,6,1,2,14.314827,1.921956,False,True,SA,1,human,6.0,NaN,NaN
1,845,50_50,5,1,2,15.054317,4.222230,False,True,SA,1,human,5.0,NaN,NaN
2,245,50_50,4,1,1,14.314827,6.508956,True,True,SA,1,human,4.0,NaN,NaN
3,72,50_50,4,2,2,8.775056,15.054317,True,True,SA,1,human,4.0,NaN,NaN
4,469,50_50,4,2,2,4.222230,19.885165,True,True,SA,1,human,4.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38995,698,100_0,present,1,1,11.159944,6.508956,True,False,gemini-2.5-pro,1,model,NaN,-15.2,-5.5
38996,910,100_0,absent,1,2,9.651833,4.986334,False,False,gemini-2.5-pro,0,model,NaN,-8.8,-3.5
38997,880,100_0,absent,2,2,4.222230,8.775056,True,False,gemini-2.5-pro,0,model,NaN,4.0,8.5
38998,53,100_0,present,2,2,4.986334,11.908300,True,False,gemini-2.5-pro,1,model,NaN,4.8,13.5


In [15]:
# Mapping side_selected numeric codes to text labels
side_map = {1: "left", 2: "right"}

df = project_master.copy()

# Create a column for the expected validity
df["expected_valid_cue"] = (df["cue_points"] == df["side_selected"].map(side_map))

# Identify rows where expected validity does NOT match actual valid_cue
mismatches = df[df["expected_valid_cue"] != df["valid_cue"]]

print("Number of mismatches:", len(mismatches))
display(mismatches.head())

# Summary check
print("\nSummary:")
print(df["expected_valid_cue"].value_counts().rename("Expected Valid Cue"))
print(df["valid_cue"].value_counts().rename("Actual Valid Cue"))


Number of mismatches: 24999


,stimID,condition,response,side_selected,cue_points,line1_angle,line2_angle,valid_cue,TP,participantID,decision,type,confidence,est1,est2,expected_valid_cue
2,245,50_50,4,1,1,14.314827,6.508956,True,True,SA,1,human,4.0,NaN,NaN,False
3,72,50_50,4,2,2,8.775056,15.054317,True,True,SA,1,human,4.0,NaN,NaN,False
4,469,50_50,4,2,2,4.222230,19.885165,True,True,SA,1,human,4.0,NaN,NaN,False
5,468,50_50,4,2,2,6.508956,17.468023,True,True,SA,1,human,4.0,NaN,NaN,False
6,923,50_50,4,1,1,13.392498,4.222230,True,True,SA,1,human,4.0,NaN,NaN,False



Summary:
expected_valid_cue
False    39000
Name: Expected Valid Cue, dtype: int64
valid_cue
True     24999
False    14001
Name: Actual Valid Cue, dtype: int64


In [11]:
# %%
# List all participant IDs
print("Participant IDs:")
print(project_master["participantID"].unique())
print()

# %%
# Count unique trials per participant: (participantID, stimID, condition)
trial_counts = (
    project_master
    .drop_duplicates(subset=["participantID", "stimID", "condition"])
    .groupby("participantID")
    .size()
    .reset_index(name="unique_trials")
    .sort_values("unique_trials", ascending=False)
)

trial_counts


Participant IDs:
['SA' 'AG' 'CY' 'JH' 'KM' 'AW' 'AZ' 'GS' 'UR' 'HG' 'KZ' 'BC'
 'gemini-2.5-pro']



,participantID,unique_trials
0,AG,3000
1,AW,3000
2,AZ,3000
3,BC,3000
4,CY,3000
5,GS,3000
6,HG,3000
7,JH,3000
8,KM,3000
9,KZ,3000


In [12]:
# %%
# Count TP=0 and TP=1 for each participant
tp_check = (
    project_master
    .groupby(["participantID", "TP"])
    .size()
    .unstack(fill_value=0)         # makes TP values into columns
    .rename(columns={0: "TP_0", 1: "TP_1"})
    .reset_index()
    .sort_values("participantID")
)

tp_check


TP,participantID,TP_0,TP_1
0,AG,1500,1500
1,AW,1500,1500
2,AZ,1500,1500
3,BC,1500,1500
4,CY,1500,1500
5,GS,1500,1500
6,HG,1500,1500
7,JH,1500,1500
8,KM,1500,1500
9,KZ,1500,1500
